# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [1]:
class Matrix:
    # 1) 
    def __init__(self, *args):
        if len(args) == 2:
            n, m = args
            self.matrix = [[0.0 for _ in range(m)] for _ in range(n)]
        elif len(args) == 1 and isinstance(args[0], list) and all(isinstance(row, list) for row in args[0]):
            self.matrix = args[0]
            self.validate_matrix()
        else:
            raise ValueError("Invalid matrix initialization")

    def validate_matrix(self):
        if not self.matrix:
            return
        row_len = len(self.matrix[0])
        for row in self.matrix:
            if len(row) != row_len:
                raise ValueError("Matrix rows must have the same length")

    # 2) 
    def __getitem__(self, index):
        if isinstance(index, tuple):
            return self.matrix[index[0]][index[1]]
        elif isinstance(index, slice):
            return Matrix(self.matrix[index])
        else:
            return self.matrix[index]

    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            self.matrix[index[0]][index[1]] = value
        else:
            self.matrix[index] = value

    # 3) 
    def shape(self):
        return len(self.matrix), len(self.matrix[0]) if self.matrix else (0, 0)

    def transpose(self):
        n, m = self.shape()
        transposed = [[self.matrix[i][j] for i in range(n)] for j in range(m)]
        return Matrix(transposed)

    def row(self, n):
        return Matrix([self.matrix[n]])

    def column(self, n):
        return Matrix([[row[n]] for row in self.matrix])

    def to_list(self):
        return self.matrix

    def block(self, n_0, n_1, m_0, m_1):
        block_matrix = [row[m_0:m_1] for row in self.matrix[n_0:n_1]]
        return Matrix(block_matrix)

    # 4)
    def scalarmul(self, c):
        return Matrix([[c * element for element in row] for row in self.matrix])

    def add(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrices must have the same shape for addition")
        return Matrix([[self.matrix[i][j] + other.matrix[i][j] for j in range(self.shape()[1])] for i in range(self.shape()[0])])

    def sub(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrices must have the same shape for subtraction")
        return Matrix([[self.matrix[i][j] - other.matrix[i][j] for j in range(self.shape()[1])] for i in range(self.shape()[0])])

    def mat_mult(self, other):
        if self.shape()[1] != other.shape()[0]:
            raise ValueError("Matrices must have compatible shapes for matrix multiplication")
        result = [[0 for _ in range(other.shape()[1])] for _ in range(self.shape()[0])]
        for i in range(self.shape()[0]):
            for j in range(other.shape()[1]):
                for k in range(self.shape()[1]):
                    result[i][j] += self.matrix[i][k] * other.matrix[k][j]
        return Matrix(result)

    def element_mult(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrices must have the same shape for element-wise multiplication")
        return Matrix([[self.matrix[i][j] * other.matrix[i][j] for j in range(self.shape()[1])] for i in range(self.shape()[0])])

    def equals(self, other):
        return self == other

    # 5) 
    def __eq__(self, other):
        if not isinstance(other, Matrix):
            return False
        return self.matrix == other.matrix

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            return self.scalarmul(other)
        elif isinstance(other, Matrix):
            return self.mat_mult(other)
        else:
            raise TypeError("Unsupported operand type(s) for *: Matrix and Matrix, or Matrix and scalar")

    def __rmul__(self, other):
        return self.__mul__(other)

    def __add__(self, other):
        return self.add(other)

    def __sub__(self, other):
        return self.sub(other)

    def __repr__(self):
        return '\n'.join(str(row) for row in self.matrix)

# 6) Standalone Matrix Functions

def constant(n, m, c):
    return Matrix([[float(c) for _ in range(m)] for _ in range(n)])

def zeros(n, m):
    return constant(n, m, 0)

def ones(n, m):
    return constant(n, m, 1)

def eye(n):
    identity = [[0.0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        identity[i][i] = 1.0
    return Matrix(identity)

# 7) Demonstrating Matrix Properties
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])
C = Matrix([[9, 10], [11, 12]])
I = eye(2)

print("1) (AB)C == A(BC):", (A * B) * C == A * (B * C))
print("2) A(B+C) == AB+AC:", A * (B + C) == A * B + A * C)
print("3) AB != BA:", A * B != B * A)
print("4) AI == A:", A * I == A)

print("5) 2 * A:", 2 * A)
print("6) A * 2:", A * 2)
print("7) A + B:", A + B)
print("8) A - B:", A - B)
print("9) A * B:", A * B)
print("10) A == A:", A == A)

D = Matrix(2,2)
D = A
print("11) D = A:", D)

print("12) A[0,1]:", A[0, 1])
print("13) A[1][1]:", A[1][1])

print("14) A.transpose():", A.transpose())
print("15) A.row(0):", A.row(0))
print("16) A.column(1):", A.column(1))
print("17) A.block(0,2,0,1):", A.block(0,2,0,1))
print("18) A.to_list():", A.to_list())


1) (AB)C == A(BC): True
2) A(B+C) == AB+AC: True
3) AB != BA: True
4) AI == A: True
5) 2 * A: [2, 4]
[6, 8]
6) A * 2: [2, 4]
[6, 8]
7) A + B: [6, 8]
[10, 12]
8) A - B: [-4, -4]
[-4, -4]
9) A * B: [19, 22]
[43, 50]
10) A == A: True
11) D = A: [1, 2]
[3, 4]
12) A[0,1]: 2
13) A[1][1]: 4
14) A.transpose(): [1, 3]
[2, 4]
15) A.row(0): [1, 2]
16) A.column(1): [2]
[4]
17) A.block(0,2,0,1): [1]
[3]
18) A.to_list(): [[1, 2], [3, 4]]


In [3]:
#Write a function make_deck that returns a list of all of the cards in a standard card deck. 
#The return should be a list of tuples of pairs of suit and value. 
#For example the 10 of Clubs would be ('Clubs', 10) and Queen of Hearts would be ('Hearts', 'Queen'). Recall that a deck has 52 cards, divided into 4 suits (Clubs, Diamonds, Hearts, and Spades), and that each suit has 13 cards: 2 to 10, Jack, Queen, King, and Ace. 




def make_deck():
    suits = ["Clubs", "Diamonds", "Hearts", "Spades"]
    values = [2, 3, 4, 5, 6, 7, 8, 9, 10, "Jack", "Queen", "King", "Ace"]
    return [(suit, value) for suit in suits for value in values]

# Example usage
deck = make_deck()
print(deck)


[('Clubs', 2), ('Clubs', 3), ('Clubs', 4), ('Clubs', 5), ('Clubs', 6), ('Clubs', 7), ('Clubs', 8), ('Clubs', 9), ('Clubs', 10), ('Clubs', 'Jack'), ('Clubs', 'Queen'), ('Clubs', 'King'), ('Clubs', 'Ace'), ('Diamonds', 2), ('Diamonds', 3), ('Diamonds', 4), ('Diamonds', 5), ('Diamonds', 6), ('Diamonds', 7), ('Diamonds', 8), ('Diamonds', 9), ('Diamonds', 10), ('Diamonds', 'Jack'), ('Diamonds', 'Queen'), ('Diamonds', 'King'), ('Diamonds', 'Ace'), ('Hearts', 2), ('Hearts', 3), ('Hearts', 4), ('Hearts', 5), ('Hearts', 6), ('Hearts', 7), ('Hearts', 8), ('Hearts', 9), ('Hearts', 10), ('Hearts', 'Jack'), ('Hearts', 'Queen'), ('Hearts', 'King'), ('Hearts', 'Ace'), ('Spades', 2), ('Spades', 3), ('Spades', 4), ('Spades', 5), ('Spades', 6), ('Spades', 7), ('Spades', 8), ('Spades', 9), ('Spades', 10), ('Spades', 'Jack'), ('Spades', 'Queen'), ('Spades', 'King'), ('Spades', 'Ace')]
